Distributed Meshes and Spaces
===

Setting up the client and the world-communiactor:

In [ ]:
from ipyparallel import Client
c = Client()
c.ids

In [ ]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD

The master generates a mesh, which is then distributed within the team of processors. The master process calls the graph partitioning library metis, which assigns a process id to each element. Then, elements are sent to the process with according rank.

The distribution is done for the Netgen mesh. Parallel uniform refinement of the Netgen-mesh is also possible.

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square

if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.1)
    print ("global num els =", len(ngmesh.Elements2D()))
    ngmesh.Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)

for l in range(0):
    ngmesh.Refine()
    
mesh = Mesh(ngmesh)
print ("process",comm.rank,"got elements:",mesh.GetNE(VOL))

Use a the collective communication `reduce` to combine data from each process to one global value. Default reduction operation is summation. Only the root process gets the result. Alternatively, use 'allreduce' to broadcast the result to all team members:

In [ ]:
%%px
sumup = comm.reduce(mesh.GetNE(VOL))
print ("summing up num els: ", sumup)

We can retriev the `mesh` variable from each node of the cluster. The master process returns the global mesh, each worker only its local part. The list 'meshes' obtains the list of meshes.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
meshes = c[:]['mesh']
for m in meshes:
    Draw (m)

Distributed Finite element spaces
===

We can define finite element spaces on the distributed mesh. 
Every process only defines dofs on each subset of elements:

In [ ]:
%%px
fes = H1(mesh, order=2)
print ("ndof local =", fes.ndof, ", ndof global =", fes.ndofglobal)

In [ ]:
%%px
sumlocdofs = comm.reduce (fes.ndof)
if comm.rank == 0:
    print ("sum of local dofs:", sumlocdofs)

The sum of local dofs is larger than the global number of dofs, since dofs at interface nodes are counted multiplel times.

We can define distributed grid-functions. Global operations like the `Integrate` function performs local integration, and sum up the result:

In [ ]:
%%px
gfu = GridFunction(fes)
gfu.Set(x*y)
print ("integrate:", Integrate(gfu, mesh))

We can retrieve the gridfunction to the local Python scope:

In [ ]:
gfus = c[:]['gfu']
print ("integrate locally:", Integrate(gfus[0], gfus[0].space.mesh))

Draw (gfus[0], min=0,max=1)
Draw (gfus[1], min=0,max=1);

We can use a piece-wise constant space to visualize the mesh distribution:

In [ ]:
%%px
gfl2 = GridFunction(L2(mesh, order=0))
gfl2.vec[:] = comm.rank

In [ ]:
Draw (c[:]['gfl2'][0])

The `ParallelDofs` class
---

The backbone of connection of dofs is the `ParallelDofs` class. It is provided by a distributed finite element space, based on the connectivity of the mesh. The pardofs object knows with which other processes the dof is shared. We can ask for all dof numbers shared with a particular process, and obtain a list of local dof nrs. The ordering is consistent for both partners. 

In [ ]:
%%px
pardofs = fes.ParallelDofs()
for otherp in range(comm.size):
    print ("with process", otherp, "I share dofs", \
           list(pardofs.Proc2Dof(otherp)))